__This notebook is used for testing the logger.__<br>
Do not grab the logger from this notebook. <br>
Instead import the following:
```py
import os
import sys
sys.path.append(os.join(os.getcwd(), 'logger'))
from Logger import Logger
```

In [ ]:
from enum import Enum

In [ ]:
class Logger():
    """
    A component used for logging events of other components.
    Designed to help increase readibility of logs by allowing the user to
    customise their own event messages and specify an urgency.
    
    Params:
        database_name: str - The name the logger will use for the logs database file (e.g. "logs_scaper.db") 
    """
    def __init__(self, database_name):
        assert type(database_name) is str, 'Database path must be of type str'
        assert len(database_name) > 0, 'Database path must be provided to logger'
        self.db_name = database_name
        
    class _Urgency(Enum):
        """
        A private enum class of the Logger component.
        Contains urgency levels along with a description of what
        the intended meaning is.
        """
        NONE     = 0 # Default value when urgency is not specified
        LOW      = 1 # Expected event and/or result
        MODERATE = 2 # Potentially unexpected event
        HIGH     = 3
        SEVERE   = 4